<a href="https://colab.research.google.com/github/tahir826/learn-agentic-ai/blob/master/langgraph/chain_reducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain-google-genai langchain_core langgraph

In [1]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.",name="Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for m in messages:
    m.pretty_print()

ModuleNotFoundError: No module named 'langchain_core'

In [ ]:
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=google_api_key)
result = llm.invoke(messages)
type(result)

langchain_core.messages.ai.AIMessage

In [ ]:
result

AIMessage(content='There\'s no single "best" place to see orcas in the US, as it depends on the time of year and what kind of viewing experience you\'re hoping for.  Orcas, or killer whales, have different populations with varying ranges.  However, some of the most popular and reliable locations include:\n\n* **Washington State (San Juan Islands):** This is arguably the most famous and consistently reliable place to see orcas in the US.  The resident orca pods, particularly the Southern Resident killer whales (J, K, and L pods), are frequently sighted here, especially during the summer months.  Whale watching tours operate frequently from Friday Harbor, Anacortes, and other locations.  The downside is that it can be crowded, and sightings aren\'t guaranteed.\n\n* **Alaska:**  Alaska boasts several orca populations, including transient orcas that hunt seals and sea lions, and resident orcas.  Specific locations like Southeast Alaska (Juneau, Ketchikan) and Prince William Sound offer exc

In [ ]:
result.response_metadata

{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'safety_ratings': []}

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools: ChatGoogleGenerativeAI = llm.bind_tools([multiply])

In [ ]:
function_call = llm_with_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")])
function_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f2dd5c69-8746-4090-bbda-eba3c7f5a404-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'b4ee8465-d200-42d8-b2e0-93e74ad82f87', 'type': 'tool_call'}], usage_metadata={'input_tokens': 22, 'output_tokens': 3, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})

In [ ]:
function_call.additional_kwargs['function_call']

{'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:

from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built
    pass

In [ ]:
# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", name="Lance")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Test
add_messages(initial_messages , new_message)

[AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}, name='Model', id='e9b0e9dd-efd4-42f0-8a9e-c60002bdd862'),
 HumanMessage(content="I'm looking for information on marine biology.", additional_kwargs={}, response_metadata={}, name='Lance', id='ed203b95-0963-4814-8271-e324f73b847a'),
 AIMessage(content='Sure, I can help with that. What specifically are you interested in?', additional_kwargs={}, response_metadata={}, name='Model', id='7c697335-cfb1-41f6-9564-12bd59f4a033')]

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder: StateGraph = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph: CompiledStateGraph  = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 4 and 2!")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
initail_messages = [AIMessage(content="Hello! how can i assist you today"),
                    HumanMessage(content="i want some basic knowledge about marine biology")]

new_messages = AIMessage(content="sure i can help you that please specify in which topic do want to assist")

add_messages = (initail_messages, new_messages)
add_messages

([AIMessage(content='Hello! how can i assist you today', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='i want some basic knowledge about marine biology', additional_kwargs={}, response_metadata={})],
 AIMessage(content='sure i can help you that please specify in which topic do want to assist', additional_kwargs={}, response_metadata={}))

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Node

def tool_calling_llm(state: MessagesState):
  return {"message":[llm_with_tools.invoke(state["messages"])]}